In [1]:
pip install numpy pandas scikit-learn tensorflow matplotlib seaborn jupyterlab torch torch-geometric rdkit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# === Core Libraries ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import glob
import gc
from tqdm import tqdm
import concurrent.futures

# === RDKit for Molecular Processing ===
from rdkit import Chem

# === PyTorch & PyTorch Geometric ===
import torch
import torch.nn as nn
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, NNConv, global_mean_pool

# === TensorFlow / Keras ===
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, GlobalMaxPooling1D,
    BatchNormalization, Dropout, Dense, Concatenate, Add, LayerNormalization
)
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.losses import MeanSquaredError

# === Scikit-learn ===
from sklearn.model_selection import train_test_split


c:\Users\NongNam\Documents\AI_Builder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# === Load data ===
df = pd.read_csv("Kd.csv")
df.head()

,SMILES,target_sequence,Kd
0,COc1cc2nc(cnc2cc1OCC#N)-c1ccc(CC(=O)Nc2cc(on2)...,MPALARDGGQLPLLVVFSAMIFGTITNQDLPVIKCVLINHKNNDSS...,8.301030
1,Cn1nc(C(=O)NC[C@@]2(O)CC[C@@H](CC2)S(C)(=O)=O)...,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,7.795880
2,Oc1ccc(/C=C/c2cc(O)cc(O)c2)cc1,MDDIYKAAVEQLTEEQKNEFKAAFDIFVLGAEDGCISTKELGKVMR...,3.614394
3,Cc1ccc(CNc2nc3c(N)ncnc3n2[C@@H]2O[C@H](CO)[C@@...,MSKGPAVGIDLGTTYSCVGVFQHGKVEIIANDQGNRTTPSYVAFTD...,6.530178
4,CC\C=C\CCOC(=O)c1cccnc1,MSALGVTVALLVWAAFLLLVSMWRQVHSSWNLPPGPFPLPIIGNLF...,5.886057


In [4]:
df.isna().sum()

SMILES             0
target_sequence    0
Kd                 0
dtype: int64

In [5]:
len(df)

55011

In [3]:
# Encode amino acid sequences
aa_list = 'ACDEFGHIKLMNPQRSTVWY'
aa_to_int = {aa: i + 1 for i, aa in enumerate(aa_list)}

def encode_protein(seq):
    return [aa_to_int.get(aa, 0) for aa in seq]

df['protein_encoded'] = df['target_sequence'].apply(encode_protein)

# Pad protein sequences
max_len_protein = 1000
X_protein = pad_sequences(df['protein_encoded'], maxlen=max_len_protein, padding='post')

# Store Kd values
y_kd = df['Kd'].values


In [4]:
def get_atom_features(atom):
    return [
        atom.GetAtomicNum(),
        atom.GetDegree(),
        atom.GetFormalCharge(),
        int(atom.GetHybridization()),
        int(atom.GetIsAromatic())
    ]

In [5]:
def get_bond_features(bond):
    return [
        int(bond.GetBondTypeAsDouble()),   # Single=1.0, Double=2.0, etc.
        int(bond.GetIsConjugated()),
        int(bond.GetIsAromatic())
    ]

In [6]:
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    smiles = Chem.MolToSmiles(mol)
    mol = Chem.MolFromSmiles(smiles)

    x = torch.tensor([get_atom_features(atom) for atom in mol.GetAtoms()], dtype=torch.float)

    edge_index = []
    edge_attr = []

    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index += [[i, j], [j, i]]
        bond_feat = get_bond_features(bond)
        edge_attr += [bond_feat, bond_feat]  # Add both directions

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# Apply and filter out None values (only valid SMILES)
graph_data_list = []
valid_indices = []

for i, smiles in enumerate(df['SMILES']):
    graph = smiles_to_graph(smiles)
    if graph is not None:
        graph_data_list.append(graph)
        valid_indices.append(i)

# Filter protein and Kd tensors to match valid molecules
X_protein = X_protein[valid_indices]
y_kd = y_kd[valid_indices]


[18:34:46] Explicit valence for atom # 22 N, 4, is greater than permitted
[18:34:51] Explicit valence for atom # 22 N, 4, is greater than permitted
[18:34:57] Explicit valence for atom # 22 N, 4, is greater than permitted
[18:35:07] Explicit valence for atom # 22 N, 4, is greater than permitted
[18:35:29] Explicit valence for atom # 22 N, 4, is greater than permitted
[18:35:41] Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20 21 22 23 25
[18:35:50] Explicit valence for atom # 22 N, 4, is greater than permitted


In [7]:
print(graph_data_list[0])

Data(x=[39, 5], edge_index=[2, 86], edge_attr=[86, 3])


In [8]:
class GNNEncoder(nn.Module):
    def __init__(self, node_in=5, edge_in=3):
        super().__init__()
        self.edge_nn1 = nn.Sequential(
            nn.Linear(edge_in, 32),
            nn.ReLU(),
            nn.Linear(32, node_in * 64)
        )
        self.conv1 = NNConv(node_in, 64, self.edge_nn1, aggr='mean')

        self.edge_nn2 = nn.Sequential(
            nn.Linear(edge_in, 32),
            nn.ReLU(),
            nn.Linear(32, 64 * 128)
        )
        self.conv2 = NNConv(64, 128, self.edge_nn2, aggr='mean')

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        x = torch.relu(self.conv1(x, edge_index, edge_attr))
        x = torch.relu(self.conv2(x, edge_index, edge_attr))
        x = global_mean_pool(x, batch)
        return x

In [11]:
# --- Protein CNN Branch ---
protein_input = Input(shape=(max_len_protein,), name="protein_input")
x = Embedding(input_dim=len(aa_to_int) + 1, output_dim=64)(protein_input)
x1 = Conv1D(128, kernel_size=9, activation='relu', padding='same')(x)
x1 = BatchNormalization()(x1)
x1 = Conv1D(256, kernel_size=9, activation='relu', padding='same')(x1)
x1 = BatchNormalization()(x1)

x2 = Conv1D(128, kernel_size=5, activation='relu', padding='same')(x)
x2 = BatchNormalization()(x2)

# Skip connection via concatenation
merged_cnn = Concatenate()([x1, x2])
x_cnn = GlobalMaxPooling1D()(merged_cnn)
x_cnn = Dropout(0.4)(x_cnn)

# --- GNN Features Input ---
gnn_input = Input(shape=(128,), name="gnn_input")

# Optional layer norm or transformation layer
gnn_proj = Dense(128, activation='relu')(gnn_input)
gnn_proj = LayerNormalization()(gnn_proj)

# --- Merge branches ---
merged = Concatenate()([x_cnn, gnn_proj])

# Fully connected layers
x = Dense(256, activation='relu')(merged)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
# x = Dense(64, activation='relu')(x)
# x = Dropout(0.2)(x)

# Output layer
output = Dense(1, activation='linear')(x)

model = Model(inputs=[protein_input, gnn_input], outputs=output)
model.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['mae'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ protein_input       │ (None, 1000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1000, 64)  │      1,344 │ protein_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 1000, 128) │     73,856 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 1000, 128) │        512 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 1000, 256) │    295,168 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 1000, 128) │     41,088 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1000, 256) │      1,024 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1000, 128) │        512 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1000, 384) │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gnn_input           │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 384)       │          0 │ concatenate[0][0] │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     16,512 │ gnn_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 384)       │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 128)       │        256 │ dense[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 512)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    131,328 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │     32,896 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 595,649 (2.27 MB)

 Trainable params: 594,113 (2.27 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [12]:
class EpochHistorySaver(Callback):
    def __init__(self, save_dir='GNN_MODEL'):
        super().__init__()
        self.save_dir = save_dir
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for key, value in logs.items():
            self.history.setdefault(key, []).append(value)

        # Save history after each epoch
        filename = f'{self.save_dir}/history_epoch_{epoch+1:02d}.pkl'
        with open(filename, 'wb') as f:
            pickle.dump(self.history, f)


In [13]:
# Save the model at every epoch with the epoch number in the filename
checkpoint_cb = ModelCheckpoint(
    filepath='GNN_MODEL/model_epoch_{epoch:02d}.h5',
    save_freq='epoch',
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

history_cb = EpochHistorySaver(save_dir='GNN_MODEL')

In [14]:
torch.cuda.empty_cache()
gc.collect()

14

In [ ]:
# Create a DataLoader for batching the graph data
batch_size = 32  # You can experiment with this size depending on your memory capacity

loader = DataLoader(graph_data_list, batch_size=batch_size, shuffle=False)

In [16]:
# Initialize the GNN encoder model
gnn_model = GNNEncoder()
gnn_model.eval()
gnn_model.cpu()  # Use CPU if GPU memory is limited

# List to store the GNN features
gnn_features_list = []

# Iterate over the DataLoader with tqdm for progress tracking
with torch.no_grad():
    for batch in tqdm(loader, desc="Extracting GNN Features"):
        gnn_features = gnn_model(batch)  # Forward pass through the GNN
        gnn_features_list.append(gnn_features.cpu().numpy())  # Store the features

# Concatenate all the features
gnn_features = np.concatenate(gnn_features_list, axis=0)

Extracting GNN Features: 100%|██████████| 1719/1719 [03:39<00:00,  7.83it/s]


In [17]:
# from tensorflow.keras.models import load_model
# model = load_model('GNN_MODEL/model_epoch_52.h5', compile=False)

In [18]:
# model.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['mae'])

In [19]:
# Train the combined model
history = model.fit(
    [X_protein, gnn_features],
    y_kd,
    epochs=100,
    batch_size=64,
    validation_split=0.18,
    callbacks=[checkpoint_cb, history_cb]
)

Epoch 1/100
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 6.8435 - mae: 1.9776
Epoch 1: saving model to GNN_MODEL/model_epoch_01.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 1452s 2s/step - loss: 6.8398 - mae: 1.9770 - val_loss: 7.8804 - val_mae: 2.3939
Epoch 2/100
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 2.6622 - mae: 1.2631
Epoch 2: saving model to GNN_MODEL/model_epoch_02.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 1391s 2s/step - loss: 2.6620 - mae: 1.2631 - val_loss: 5.1337 - val_mae: 1.9252
Epoch 3/100
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 2.2719 - mae: 1.1483
Epoch 3: saving model to GNN_MODEL/model_epoch_03.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 1464s 2s/step - loss: 2.2718 - mae: 1.1482 - val_loss: 3.0808 - val_mae: 1.3575
Epoch 4/100
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 2.1302 - mae: 1.1114
Epoch 4: saving model to GNN_MODEL/model_epoch_04.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 1397s 2s/step - loss: 2.1302 - mae: 1.1114 - val_loss: 2.8568 - val_mae: 1.2940
Epoch 5/100
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 2.0259 - mae: 1.0764
Epoch 5: saving model to GNN_MODEL/model_epoch_05.h5


...

705/705 ━━━━━━━━━━━━━━━━━━━━ 1281s 2s/step - loss: 1.2959 - mae: 0.8289 - val_loss: 1.3269 - val_mae: 0.8061
Epoch 49/100
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 1.2901 - mae: 0.8264
Epoch 49: saving model to GNN_MODEL/model_epoch_49.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 1293s 2s/step - loss: 1.2902 - mae: 0.8264 - val_loss: 1.3749 - val_mae: 0.8793
Epoch 50/100
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 1.2784 - mae: 0.8225
Epoch 50: saving model to GNN_MODEL/model_epoch_50.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 1296s 2s/step - loss: 1.2784 - mae: 0.8225 - val_loss: 1.3369 - val_mae: 0.8143
Epoch 51/100
705/705 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 1.2837 - mae: 0.8201
Epoch 51: saving model to GNN_MODEL/model_epoch_51.h5


705/705 ━━━━━━━━━━━━━━━━━━━━ 1270s 2s/step - loss: 1.2837 - mae: 0.8201 - val_loss: 1.3217 - val_mae: 0.8112
Epoch 52/100
276/705 ━━━━━━━━━━━━━━━━━━━━ 12:34 2s/step - loss: 1.3349 - mae: 0.8418

KeyboardInterrupt: 

In [1]:
# Load all history files
history_files = sorted(glob.glob('GNN_MODEL/history_epoch_*.pkl'))

full_history = {}
for file in history_files:
    with open(file, 'rb') as f:
        epoch_history = pickle.load(f)
    for key, values in epoch_history.items():
        full_history.setdefault(key, []).extend(values[len(full_history.get(key, [])):])

# Plot


plt.plot(full_history['val_loss'])
plt.plot(full_history['loss'])
plt.title('Model accuracy over all epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()


NameError: name 'glob' is not defined